In [2]:
import pandas as pd
import ssl
ssl._create_default_https_context=ssl._create_unverified_context

wine=pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [4]:
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()

In [5]:
# 검증 세트

from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target=train_test_split(
    data, target, test_size=0.2, random_state=42)

In [6]:
sub_input, val_input, sub_target, val_target=train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [7]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [8]:
from sklearn.tree import DecisionTreeClassifier

dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [10]:
# 교차 검증

from sklearn.model_selection import cross_validate

scores=cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00695109, 0.00698924, 0.00800729, 0.00800276, 0.00798321]), 'score_time': array([0.00099754, 0.00099802, 0.00096989, 0.        , 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [11]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [12]:
from sklearn.model_selection import StratifiedKFold

scores=cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [14]:
splitter=StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores=cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [16]:
# 그리드 서치
# 하이퍼파라미터 튜닝

from sklearn.model_selection import GridSearchCV

params={'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [17]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)


In [18]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [21]:
dt=gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [22]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [23]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [24]:
best_index=np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [26]:
pramas={'min_inpurity_decreas':np.arange(0.0001, 0.001, 0.0001),
    'max_depth': range(5, 20, 1),
    'min_samples_split': range(2, 100, 10)}

In [27]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [28]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [29]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8681929740134745


In [31]:
# 랜덤 서치

from scipy.stats import uniform, randint

In [32]:
rgen=randint(0, 10)
rgen.rvs(10)

array([1, 8, 3, 1, 8, 6, 9, 4, 8, 9], dtype=int64)

In [33]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([ 87,  96,  84, 107,  98, 111,  98, 112, 109,  98], dtype=int64))

In [34]:
ugen=uniform(0, 1)
ugen.rvs(10)

array([0.86865609, 0.77472651, 0.8478434 , 0.76602949, 0.29541496,
       0.86285502, 0.92608357, 0.785374  , 0.53363071, 0.70963155])

In [41]:
prams={'min_impurity_decrease':uniform(0.0001, 0.001),
    'max_depth': randint(20, 50),
    'min_samples_split': randint(2,25),
    'min_samples_leaf': randint(1, 25)}

In [42]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 5 is smaller than n_iter=100. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'min_impurity_decrease': [0.0001,
                                                                  0.0002,
                                                                  0.0003,
                                                                  0.0004,
                                                                  0.0005]},
                   random_state=42)

In [43]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [44]:

print(np.max(gs.cv_results_['mean_test_score']))

0.8681929740134745


In [45]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.8653846153846154
